In [25]:
# ! pip install pinecone-client
import json
import pinecone

config = json.load(open("config.json"))
pc = pinecone.Pinecone(api_key=config['pinecone_api_key'])

In [26]:
# call the hf inference endpoint for bge en small v1.5
import requests

model_id = "BAAI/bge-small-en-v1.5"
hf_token = config['hf_token']

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}


def hf_embed(texts):
    response = requests.post(
        api_url,
        headers=headers,
        json={"inputs": texts, "options": {"wait_for_model": True}},
    )
    return response.json()

# response = hf_embed(["hello world", "this is a test"])

In [64]:
# ! pip install --upgrade openai
import openai

# openai.api_key = config['openai_api_key']

client = openai.Client(api_key=config['openai_api_key'])

def prompt_mini(prompt, history=[]):
    msg = [
            {
                "role": "system",
                "content": "You are an academic specialist working in an elite American college, eg Cornell. You specialize in analyzing data and giving expert advices for inquiring students (courses, outcomes, professors, research, reviews). Be warm, friendly, professional, and helpful. OUTPUT MUST BE IN A JSON COMPATIBLE FORMAT according to further prompts.",
            }
        ] + history + [
            {
                "role": "user",
                "content": prompt,
            },
        ]
    print(msg)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=msg
    )
    # print(completion.choices)
    if completion.choices[0].finish_reason == "length":
        return completion.choices[0].message.content, "TOO LONG"
    return completion.choices[0].message.content

In [65]:
response = prompt_mini("tell me about yourself. who are you and what are you good at?")
print(response)

[{'role': 'system', 'content': 'You are an academic specialist working in an elite American college, eg Cornell. You specialize in analyzing data and giving expert advices for inquiring students (courses, outcomes, professors, research, reviews). Be warm, friendly, professional, and helpful. OUTPUT MUST BE IN A JSON COMPATIBLE FORMAT according to further prompts.'}, {'role': 'user', 'content': 'tell me about yourself. who are you and what are you good at?'}]


APIConnectionError: Connection error.

In [97]:
UNDERSTAND_QUERY_PROMPT = """
USE INFORMATION FROM PREVIOUS MESSAGES ALONG WITH THE BELOW QUERY:
A student/user asked (query): {query}

Come up with subquestions to ask the student to clarify the query, is there something specific enough so we know a general direction? It's ok to make inferences/best guesses.
We currently can search in (namespaces):
- professors: goals, subdomains, experience, teaching
- courses: contents, outcomes, perceived_prerequisites
Make sure we gathered enough information in any (one or more) of the above eight categories.

if informative enough or the user is somewhat firm and wants that recommendation, proceed to add the following fields to JSON output: (MAKE YOUR BEST JUDGEMENT)
- index: STRING of either 'professor', 'course', or 'program' to indicate the type of recommendation the user seeks.
- rephrasing: LIST of {n} STRINGS to rephrase the query into a short declarative sentence. You know exactly what kind of course, professor, or program and can easily translate the query into a list of concrete and specific sentences. Examples: professor: "Application in separations, solar energy, and biomolecule delivery", course: "Analysis of bias in artistic and architectural expressions" or course: "Basic understanding of U.S. history". WRITE IN PERSPECTIVE OF THE FACULTY DESCRIBING WHAT THEY OFFER.
- searches: LIST of STRINGs (under professors's or courses's namespaces) that are relevant to the rephrasing. MUST BE ONE OF THE 3 or 4 namespaces.
{{ "index": "<PROFESSOR/COURSE/PROGRAM>", "rephrasing": [<REPHRASINGS>], "searches": [<SEARCHES>] }}

rephrasing shouldn't be too focused on the details, but rather on the general idea and the main goal as can be outlined in a catalog / description.

if requires more inputs and the student's tone is still warm & friendly, provide a response to the user of the form:
{{ "index": "<PROFESSOR/COURSE/PROGRAM/NONE>", "response": "<RESPONSE>" }}
to ask what information should be needed.

REMEMBER YOUR DUTY IS TO HELP THE STUDENT, AND IF THE STUDENT DOESN'T SOUND LIKE THEY'RE HAPPY, MAKE YOUR BEST JUDGEMENT to GIVE OUT the REPHRASING!

OUTPUT without ```json and ```:
"""

history = [
    {
        "role": "user",
        "content": "what is the best course to take for a beginner in computer science?",
    },
    {
        "role": "assistant",
        "content": "To better assist you in finding the best beginner course in computer science, could you clarify a few things? Are you looking for a specific area within computer science, such as programming, data science, or web development? Do you have any particular interests or career goals in mind that might influence your choice? Additionally, do you prefer online courses, in-person classes, or a hybrid approach?",
    },
    {
        "role": "user",
        "content": "I want to learn more about functional programming in Python. Any suggestions?",
    },
    {
        "role": "assistant",
        "content": "To provide you with the best suggestions for learning functional programming in Python, could you please clarify a few points? 1. Do you have any prior experience with programming or Python? 2. Are you looking for specific applications of functional programming, such as data analysis, web development, or software engineering? 3. Would you prefer a course with a hands-on approach or more theoretical content? Your answers will help me find the most suitable options for you!",
    },
    {
        "role": "user",
        "content": "just give me some hands on courses for data analysis",
    },
    {
        "role": "assistant",
        "content": "To help you find the best hands-on courses for data analysis, could you please clarify a few points? 1. What level of prior knowledge do you have in data analysis or statistics? 2. Are you interested in any specific tools or software (e.g., Python, R, Excel)? 3. What is your primary goal in learning data analysis (e.g., career change, academic interest, personal project)? Your input will help me provide more tailored recommendations!",
    },
    {
        "role": "user",
        "content": "ive already told you what i want, python data analysis, have a bit of internship experience, and want more college teachings",
    },
    {
        "role": "assistant",
        "content": "To better assist you in finding the best hands-on courses for Python data analysis, could you clarify a few things? Are you looking for a specific area within data analysis, such as machine learning, data visualization, or statistical analysis? Do you have any particular interests or career goals in mind that might influence your choice? Additionally, do you prefer online courses, in-person classes, or a hybrid approach?",
    },
]

# {
#     "index": "course",
#     "response": "To better assist you in finding the best beginner course in computer science, could you clarify a few things? Are you looking for a specific area within computer science, such as programming, data science, or web development? Do you have any particular interests or career goals in mind that might influence your choice? Additionally, do you prefer online courses, in-person classes, or a hybrid approach?",
# }

r = prompt_mini(UNDERSTAND_QUERY_PROMPT.format(
    query="make your best call, i already know math and industry experience",
    n=3
), history)
print(r)

# make your best call
# {
#     "index": "course",
#     "rephrasing": [
#         "Hands-on course for Python data analysis",
#         "Beginner-friendly data analysis course using Python",
#         "Practical approach to learning data analysis in Python",
#     ],
#     "searches": ["contents", "outcomes"],
# }

# make your best call, i want a good career in data analysis
# {
#     "index": "course",
#     "rephrasing": [
#         "Courses focused on practical data analysis using Python",
#         "Hands-on Python data analysis courses for a successful career",
#         "Courses to enhance data analysis skills specifically with Python",
#     ],
#     "searches": ["contents", "outcomes", "perceived_prerequisites"],
# }


# {
#     "index": "course",
#     "rephrasing": [
#         "Hands-on Python data analysis course for students with some industry experience",
#         "Advanced courses in Python data analysis leveraging existing math knowledge",
#         "Practical data analysis using Python for those with internship experience",
#     ],
#     "searches": ["contents", "outcomes"],
# }

# running this through our embedding vectorstore
# {
#     "index": "course",
#     "rephrasing": [
#         "Hands-on course in Python data analysis for those with industry experience and a strong math background",
#         "Advanced Python data analysis course designed for students with some internship experience",
#         "Practical-focused data analysis using Python, ideal for learners with prior knowledge and experience",
#     ],
#     "searches": ["contents", "outcomes", "perceived_prerequisites"],
# }

# {
#     "index": "course",
#     "rephrasing": [
#         "Hands-on course focusing on data analysis using Python.",
#         "Practical applications of Python in data analysis.",
#         "Advanced data analysis techniques using Python for industry professionals.",
#     ],
#     "searches": [
#         "Python for Data Analysis",
#         "Advanced Python Data Analysis Techniques",
#         "Data Visualization with Python",
#     ],
# }

[{'role': 'system', 'content': 'You are an academic specialist working in an elite American college, eg Cornell. You specialize in analyzing data and giving expert advices for inquiring students (courses, outcomes, professors, research, reviews). Be warm, friendly, professional, and helpful. OUTPUT MUST BE IN A JSON COMPATIBLE FORMAT according to further prompts.'}, {'role': 'user', 'content': 'what is the best course to take for a beginner in computer science?'}, {'role': 'assistant', 'content': 'To better assist you in finding the best beginner course in computer science, could you clarify a few things? Are you looking for a specific area within computer science, such as programming, data science, or web development? Do you have any particular interests or career goals in mind that might influence your choice? Additionally, do you prefer online courses, in-person classes, or a hybrid approach?'}, {'role': 'user', 'content': 'I want to learn more about functional programming in Python

In [98]:
embeddings = hf_embed(json.loads(r)['rephrasing'])

In [78]:
embeddings[0]

[-0.05356299877166748,
 -0.0170073714107275,
 0.017181314527988434,
 -0.03677484020590782,
 0.037204910069704056,
 -0.04091985523700714,
 -0.029377872124314308,
 0.0550694540143013,
 -0.0502801276743412,
 -0.03586249798536301,
 -0.010340052656829357,
 -0.05501973256468773,
 0.007437839172780514,
 0.003619597526267171,
 0.053977515548467636,
 -0.00013587099965661764,
 0.026347098872065544,
 0.004527593031525612,
 0.039450064301490784,
 -0.027273545041680336,
 -0.039466142654418945,
 -0.010042990557849407,
 -0.051997341215610504,
 -0.050753332674503326,
 0.007588827051222324,
 0.048535000532865524,
 0.0036418461240828037,
 -0.0720917358994484,
 -0.058449067175388336,
 -0.1539992094039917,
 0.005296823102980852,
 -0.00917502585798502,
 0.05557708814740181,
 -0.0013235785299912095,
 0.031903497874736786,
 0.021588869392871857,
 -0.0032691515516489744,
 0.006415993440896273,
 -0.0243071299046278,
 -0.023397907614707947,
 0.035230837762355804,
 0.0044714403338730335,
 -0.024669956415891647,


In [103]:
courses = pc.Index("courses")
qr = courses.query(
    namespace='contents',
    vector=embeddings[1],
    top_k=3
)
qr

{'matches': [{'id': '41913:5', 'score': 0.926129758, 'values': []},
             {'id': '31532:6', 'score': 0.912601411, 'values': []},
             {'id': '7291:4', 'score': 0.879041672, 'values': []}],
 'namespace': 'contents',
 'usage': {'read_units': 5}}

In [118]:
ids = [m['id'].split(':')[0] for m in qr['matches']]
ids, qr['matches']

import sqlite3
conn = sqlite3.connect("roster_reviews.sqlite.db")
cursor = conn.cursor()

all_courses = cursor.execute("SELECT * FROM courses WHERE course_id IN ({})".format(','.join(['?']*len(ids))), ids).fetchall()

titles = [c[3] for c in all_courses]
titles

# first embedding: ['INFO 5001', 'AEM 2841', 'ENGRG 1060']
# second: ['ECE 2400', 'STSCI 5520', 'ENGRG 1060']
# third: ['GDEV 7970', 'BEE 1050', 'ENGRG 1060']

# first: ['HADM 3710', 'ASTRO 6525', 'PLPPM 6380']
# second: ['HADM 3710', 'ASTRO 6525', 'PLPPM 6380']
# third: ['ASTRO 6525', 'PLPPM 6380', 'AEP 3640']

# first: ['AEM 5840', 'ASTRO 6525', 'PLPPM 6380']
# second: ['AEM 2841', 'ASTRO 6525', 'PLPPM 6380']
# third: ['HADM 3710', 'ASTRO 6525', 'PLPPM 6380']

# first: ['AEM 2841', 'ASTRO 6525', 'PLPPM 6380']
# second: ['HADM 3710', 'ASTRO 6525', 'PLPPM 6380']
# third: ['HADM 3710', 'ASTRO 6525', 'PLPPM 6380']

['CEE 3101',
 'CEE 5101',
 'CS 5820',
 'ENMGT 3101',
 'ENMGT 5101',
 'PHYS 1013',
 'AEM 2831',
 'ENMGT 5080',
 'DEA 2200',
 'STSCI 5520',
 'ASTRO 6525',
 'ASTRO 4523',
 'PAM 4960',
 'PAM 5060',
 'EAS 3411']

In [105]:
e = hf_embed("Programming and problem solving using Python")

In [117]:
qr = courses.query(
    namespace='outcomes',
    vector=e,
    top_k=15
)

In [114]:
result = cursor.execute("SELECT * FROM courses WHERE name ='CS 1110' and type='Lecture'").fetchall()
len(result)

44